In [104]:
using Pkg
Pkg.activate(".")
using BenchmarkTools
import Markdown 
Base.showable(::MIME"text/markdown", ::Markdown.MD) =false # same function as below
include("bnc_initialize.jl")

  Activating project at `~/Realizibility_index/bnc_julia`


show_qK_space_constrains (generic function with 1 method)

In [130]:
L = [1 0 1 10
    0 1 1  1]
model = Bnc(L=L)
[model.L
model.N]

4×4 Matrix{Int64}:
  1  0   1  10
  0  1   1   1
  1  1  -1   0
 10  1   0  -1

We especially care about while it will form: 
$$\begin{bmatrix}
0&0&0&1\\
0&0&1&0
\end{bmatrix}$$
$$\begin{bmatrix}
0&0&y&x\\
0&0&z&0
\end{bmatrix}$$

In [131]:
logK1 = 0
logK2_rg = (-6,6)
logE_rg = (-6,6)
logS_rg = (-6,6)

total_points = 1000000
point_num_eachdim = round(Int, total_points^(1/3))

#idx1: logE_min ~logE_max
#idx2: logS_min ~logS_max
#idx3: logK2_min ~logK2_max

#Initialize the array
logx_mt = Array{Vector{Float64},3}(undef, point_num_eachdim, point_num_eachdim, point_num_eachdim)

#Fill the first axis
start_logqK = [logE_rg[1], logS_rg[1], logK1 , logK2_rg[1]]
end_logqK = [logE_rg[2], logS_rg[1], logK1 , logK2_rg[1]]
logx_mt[:,1,1] .= x_traj_with_qK_change(model,start_logqK, end_logqK;input_logspace=true, output_logspace=true,saveat = range(0,1,length=point_num_eachdim))[2]

#Fill the first surface
logE_step = range(logE_rg...,length=point_num_eachdim)
for i in 1:point_num_eachdim
    start_logqK = [logE_step[i], logS_rg[1], logK1 , logK2_rg[1]]
    end_logqK = [logE_step[i], logS_rg[2], logK1 , logK2_rg[1]]
    logx_mt[i,:,1] .= x_traj_with_qK_change(model, start_logqK, end_logqK; startlogx = logx_mt[i,1,1],input_logspace=true, output_logspace=true ,saveat = range(0,1,length=point_num_eachdim))[2]
end

#Fill the whole cube
logS_step = range(logS_rg...,length=point_num_eachdim)
for i in 1:point_num_eachdim
    for j in 1:point_num_eachdim
        start_logqK = [logE_step[i], logS_step[j], logK1 , logK2_rg[1]]
        end_logqK = [logE_step[i], logS_step[j], logK1 , logK2_rg[2]]
        logx_mt[i,j,:] .= x_traj_with_qK_change(model, start_logqK, end_logqK; startlogx = logx_mt[i,j,1],input_logspace=true, output_logspace=true ,saveat = range(0,1,length=point_num_eachdim))[2]
    end
end


In [132]:
get_vertices(model, singular=false, real=false, return_idx = false)

Start finding all vertices, it may takes a while.
Done, with 8 vertices found and 7 real vertices.


1-element Vector{Vector{Int8}}:
 [4, 3]

In [133]:
x_rgm_asym = logx_mt .|> x-> assign_vertex_x(model,x,input_logspace=true,asymptotic=true) |> x->get_idx(model,x)
qK_rgm_asym = logx_mt .|> x-> assign_vertex_qK(model,x,input_logspace=true,asymptotic=true) |> x->get_idx(model,x)
x_rgm = logx_mt .|> x-> assign_vertex_x(model,x,input_logspace=true,asymptotic=false) |> x->get_idx(model,x)
qK_rgm = logx_mt .|> x-> assign_vertex_qK(model,x,input_logspace=true,asymptotic=false ) |> x->get_idx(model,x)

logK2_step = range(logK2_rg...,length=point_num_eachdim)
(minval,maxval) = values(model.vertices_idx) |> x->(minimum(x),maximum(x))

(1, 8)

In [134]:
Set(x_rgm[:])|> collect|>sort

8-element Vector{Int64}:
 1
 2
 3
 4
 5
 6
 7
 8

In [135]:
Set(qK_rgm[:]) |>collect |>sort

6-element Vector{Int64}:
 1
 2
 3
 4
 6
 7

In [136]:
idx = 3
perm = get_perm(model,idx)
is_real = get_vertex!(model,idx).real
singularity = get_singularity!(model,idx)
println("idx=$idx,perm=$perm, is_real=$is_real, singularity=$singularity")

Start calculating vertex neighbor matrix, It may takes a while.
Done.
idx=3,perm=Int8[1, 4], is_real=true, singularity=0


In [1]:
using ImageFiltering
function find_bounds(lattice)
    col_asym_x_bounds = imfilter(lattice, Kernel.Laplacian(), "replicate") # findboundary
    edge_map = col_asym_x_bounds .!= 0
    return edge_map
end

LoadError: ArgumentError: Package ImageFiltering not found in current path.
- Run `import Pkg; Pkg.add("ImageFiltering")` to install the ImageFiltering package.

In [137]:
using GLMakie

# 参数
nslices = 11   # 可以随便改 slice 数目
slice_positions = round.(Int, range(1, point_num_eachdim, length=nslices+2)[2:end-1])

f = Figure(size=(600*nslices, 2000))

cmap = :rainbow
nlevels = maxval - minval + 1
cmap_disc = cgrad(cmap, nlevels, categorical=true)
crange = (minval, maxval)

# --- 3D 图 ---
p11 = Axis3(f[1, 1])
p11.title = "x space non-asym"
volume!(p11, x_rgm; colormap=cmap, colorrange=crange)

p12 = Axis3(f[2, 1])
p12.title = "qK space non-asym"
volume!(p12, qK_rgm; colormap=cmap, colorrange=crange)

p13 = Axis3(f[3, 1])
p13.title = "x space asym"
volume!(p13, x_rgm_asym; colormap=cmap, colorrange=crange)

p14 = Axis3(f[4, 1])
p14.title = "qK space asym"
volume!(p14, qK_rgm_asym; colormap=cmap, colorrange=crange)

# --- slices 横向排列 ---
for (i, idx) in enumerate(slice_positions)
    # x space slice
    ax = Axis(f[1, i+1])
    ax.title = "x slice $i,with logK2 = $(round(logK2_step[idx],digits=2))"
    heatmap!(ax, logE_step, logS_step, x_rgm[:, :, idx]; colormap=cmap, colorrange=crange)

    # qK space slice
    ax2 = Axis(f[2, i+1])
    ax2.title = "qK slice $i,with logK2 = $(round(logK2_step[idx],digits=2))"
    heatmap!(ax2, logE_step, logS_step, qK_rgm[:, :, idx]; colormap=cmap, colorrange=crange)

    ax3 = Axis(f[3, i+1])
    ax3.title = "x slice asym $i,with logK2 = $(round(logK2_step[idx],digits=2))"
    heatmap!(ax3, logE_step, logS_step, x_rgm_asym[:, :, idx]; colormap=cmap, colorrange=crange)

    ax4 = Axis(f[4, i+1])
    ax4.title = "qK slice asym $i,with logK2 = $(round(logK2_step[idx],digits=2))"
    heatmap!(ax4, logE_step, logS_step, qK_rgm_asym[:, :, idx]; colormap=cmap, colorrange=crange)
end

# label
for p in [p11, p12]
    p.xlabel = "logE"
    p.ylabel = "logS"
    p.zlabel = "logK2"
end
# for ax in content(f)
#     if ax isa Axis
#         ax.xlabel = "logE"
#         ax.ylabel = "logS"
#     end
# end

# colorbar
Colorbar(f[:, end+1], colorrange=crange, colormap=cmap_disc, ticks=minval:maxval)
# save("rgm_slices_and_volume.png", f)
display(f)


GLMakie.Screen(...)

In [68]:
for index in  1:length(find_all_vertices!(model))
    vtx = get_vertex!(model,index)
    perm = vtx.perm
    singularity = vtx.singularity
    real = vtx.real
    println("Vertex $index: perm=$perm, singularity=$singularity, real=$real")
end

Vertex 1: perm=Int8[1, 2], singularity=0, real=true
Vertex 2: perm=Int8[1, 3], singularity=0, real=true
Vertex 3: perm=Int8[1, 4], singularity=0, real=true
Vertex 4: perm=Int8[3, 2], singularity=0, real=true
Vertex 5: perm=Int8[3, 3], singularity=1, real=true
Vertex 6: perm=Int8[4, 2], singularity=0, real=true
Vertex 7: perm=Int8[4, 3], singularity=0, real=false
Vertex 8: perm=Int8[4, 4], singularity=1, real=true
